In [1]:
import torchaudio
torchaudio.set_audio_backend("soundfile")

from datasets.Sampler import Sampler
from graphs.models.VGGVox import VGGVox

from agents.trainer import Trainer


from utils.config import *

# JSON = 'configs/VGGVox.json'
# JSON = 'configs/pre_trained/AutoSpeech.json'
# JSON = 'configs/pre_trained/ResNetSE34L.json'
JSON = 'configs/pre_trained/Cnn3D.json'
# JSON = 'configs/Test.json'

config = process_config(JSON)

[INFO]: Experiment: Cnn3D Pretrained.


In [2]:
trainer = Trainer(config)

[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: __Python VERSION:  3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:22:33) [MSC v.1916 64 bit (AMD64)]
[INFO]: __pyTorch VERSION:  1.9.0
[INFO]: __CUDA VERSION
[INFO]: __CUDNN VERSION:  7605
[INFO]: __Number CUDA Devices:  1
[INFO]: __Devices
[INFO]: Active CUDA Device: GPU 0
[INFO]: Available devices  1
[INFO]: Current cuda device  0


In [3]:
trainer.run()

Epoch-1 Validation | VEER 27.2534% MDC 0.90111 | Efficiency 73.83% : 100%|██████████| 37720/37720 [1:12:36<00:00,  8.66it/s]
[INFO]: Validation at epoch-1 completed in 4356.0s (8.7samples/s). VEER 27.2534% MDC 0.90111 


In [4]:
total_params = 0
for name, parameter in trainer.__model__.named_parameters():
    if not parameter.requires_grad:
        continue
    param = parameter.numel()
    total_params += param
print(f"Total Trainable Params: {total_params}")

Total Trainable Params: 1939995
